In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tf_explain.core.grad_cam import GradCAM


In [2]:
# Dataset paths
train_dir = r"D:\university\cse400(A,B,C)\Dataset\datasetM\train"
test_dir = r"D:\university\cse400(A,B,C)\Dataset\datasetM\test"

# Parameters
img_height = 224
img_width = 224
batch_size = 32
num_classes = len(os.listdir(train_dir))  # Assuming each subdirectory is a class


In [3]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load datasets
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 25255 images belonging to 16 classes.
Found 17547 images belonging to 16 classes.


In [4]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [6]:
# Train the model
history = model.fit(
    train_data,
    epochs=200,
    validation_data=test_data
)






Epoch 1/200
790/790 [==============================] - 266s 337ms/step - loss: 2.1561 - accuracy: 0.2988 - val_loss: 3.1507 - val_accuracy: 0.1723
Epoch 2/200
790/790 [==============================] - 270s 341ms/step - loss: 2.1368 - accuracy: 0.3029 - val_loss: 4.0579 - val_accuracy: 0.1734
Epoch 3/200
790/790 [==============================] - 245s 310ms/step - loss: 2.1245 - accuracy: 0.3083 - val_loss: 3.3656 - val_accuracy: 0.1771
Epoch 4/200
790/790 [==============================] - 247s 312ms/step - loss: 2.1030 - accuracy: 0.3150 - val_loss: 3.4529 - val_accuracy: 0.1849
Epoch 5/200
790/790 [==============================] - 243s 308ms/step - loss: 2.0905 - accuracy: 0.3197 - val_loss: 3.6987 - val_accuracy: 0.1848
Epoch 6/200
790/790 [==============================] - 244s 309ms/step - loss: 2.0824 - accuracy: 0.3225 - val_loss: 3.1932 - val_accuracy: 0.1829
Epoch 7/200
790/790 [==============================] - 243s 308ms/step - loss: 2.0640 - accuracy: 0.3308 - val_loss: 3

KeyboardInterrupt: 

In [7]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.2f}")


549/549 [==============================] - 90s 164ms/step - loss: 3.5283 - accuracy: 0.3076
Test Accuracy: 0.31


In [8]:
# Load a test image
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Get a batch of test data
test_images, test_labels = next(test_data)

# Select a single image for Grad-CAM
test_image = test_images[0]
test_image_array = np.expand_dims(test_image, axis=0)

# Grad-CAM Explanation
explainer = GradCAM()
explanation = explainer.explain(
    validation_data=(test_image_array, None),
    model=model,
    layer_name='conv2d_2'  # Specify the last convolutional layer
)

# Visualize the explanation
plt.imshow(test_image)
plt.title("Original Image")
plt.show()

plt.imshow(explanation, cmap='jet', alpha=0.6)
plt.title("Grad-CAM Heatmap")
plt.show()


TypeError: GradCAM.explain() missing 1 required positional argument: 'class_index'